#  1.  Reinforcement Learning: the gist

## 1.1  High level overview



## 1.2 Common Reinforcement Learning problems

To get a more intuitive sense of what these common components of Reinforcement Learning problems are, lets look at a variety of examples.

### Example 1:  Grid world: making a shortest-path finding AI

Grid world is a simplified version of what is called the shortest path problem - a problem often solved in video games (this is how enemy AI units find you in the game), mapping services (finding the shortest route from A to B), and robot path planning (e.g., for a cleaning robot) - get a user from its starting location to a desired destination in as few steps as possible.    

There are many algorithms specifically designed to solve just this task - the most popular being [Dijkstra’s and A* algorithms](http://www.redblobgames.com/pathfinding/a-star/introduction.html).  However here we will use the flexible RL framework as it too provides great results, and the (relative) simplicity of the problem will allow us to illustrate the more general RL learning process hopefully clearly.

In this toy problem we have a grid like the one illustrated below.  Each square tile is a location in the world.  Here the black square denotes the user, the green square the desired destination, and the blue squares impenetrable obstacles (the user cannot move to).  

The actions available to our user are to move one unit left, right, up, or down, or stay still, and it can move to any free square (here colored magenta) or the goal (colored green).  If the user ever moves to the goal square the game is over.

In [11]:
from my_gridworld import my_gridworld
test = my_gridworld()
test.color_grid()

,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


###  Example 2:  Making an AI that can play chess

Another problem RL can be applied to is making an intelligent agent to play (and win) the game of chess.

<img src="images/chess.png",width=250,height=250>

### Example 3:   An AI that can balanace a pole on a moving cart (a.k.a. the inverted pendulum)

A classic toy control problem  - teach an agent how to balance a pole on a moving cart.  The pole is free to rotate about an axis on the cart, with the cart on a track so that it may be moved left and right - affecting the location of the pole.  The pole feels the force of Earth's gravity, and so if unbalanced will fall to the ground.  

<img src="images/cartpole.png",width=300,height=300>

This problem is a prototype for more general problems in mechanical control and robotics.

## 1.3  The fundamental components of an RL problem

Lets define the modeling components of a standard Reinforcement Learning problem - afterwards we will return to our example problems and describe the specific instances of these components for each example.  

**There are *four* main components to the RL model: states, features, actions, and rewards.**  These are - at a high level:

1\.    A **state** is a specific configuration of all objects in a problem's enviroment.  A **state space** is the collection of all possible states this enviroment is capable of producing.  The state space is defined by the problem we aim to solve.

- In *grid world* any state consists of one piece of information: the current location of the player (the black square).  The state space is all possible locations on the board where the player can go.


- In the chess problem a state is any (legal) configuration of the white and black pieces on the board (and some pieces may be missing).  The state space is the set of all possible configurations of chess pieces on the board.  The state space is defined by the problem we aim to solve.


- In the cart-pole problem a state is a complete set of information about the cart and pole's position.  This includes: the cart position, the cart velocity, the angle of the pole measured as its deviation from the vertical position, and the angular velocity of the pole.  While these are technically continuous values, in practice they are finelly discretized.  The state does **not** include technical / physical information about the enviroment e.g., the fact that gravity exists, its precise force, etc.,


2\.  The agent can then take an **action** - the set of actions is either determined by us or by the problem environment.  The **action space** is the total set of such available actions.  


- For grid world, a toy problem, we can choose a set of actions.  For example we could decide that the agent can only move the black square adjacently one unit up/down/left/right or keep it still.  Alternatively, we could allow the agent to move the black square diagonally as well.


- In the chess problem an action is a legal move of any of its current pieces on the board.


- In the cart pole problem - as in robotics applications as well - the range of actions is completely defined by the available range of motions of the machine being directly controlled.  In cart pole the agent directly controls the cart, which in 2-dimensions can only move left or right.


3\.  Once this action is taken the agent receives a **reward** - or a signal that the agent has performed a good or bad action based on our desired goal.  We (humans) decide on the reward structure (in order to induce our agent to accomplish our desired goal).  We want a reward for a given action to be **larger** for those actions that get us closer to accomplishing our goal (and less for those actions which do not), or vice versa.  To reiterate: this is the (only) way we communicate our desired goal to the agent.

- For grid world we can assign a negative value like -1 to all actions (one unit movement) which leads to a non-goal state, and a large positive number like 1000 to actions leading to the goal state itself.  However these exact numbers are arbitrary, so long as a negative reward is given to actions not directly leading to the goal (and a positive reward is given when the goal square is reached).
 
 
- For the chess problem one reward structure to induce our agent to learn how to win could be as follows: any move made that does not check mate the opponent recieves -1 reward, a move that sucessfully check mates the oponnent receieves a reward of 10000


- For the cart-pole problem one common choice of reward structure - at every state at which the angle of the pole is above a certain threshold the reward is 1, otherwise it is 0.

##  1.4  How the four RL pieces fit together

These four pieces connect together to create a feedback system that - when properly engineered - allow us to train an AI agent to accomplish a desired goal via maximization of the surrogate rewards.  In order to derive the RL algorithms that allow for this we need to first represent each of the components algebraically.

### Problem solving in steps and RL component notation

For now suppose a for a given problem that there are only $N$ possible states and $M$ available actions.  

- In the grid world problem the number of states is finite and equal to the number of places the player can travel.


- In the chess problem the number of states is equal to the total number of possible configurations of the board, while the number of actions is equal to the number of legal moves available to the pieces on the board.


- In the cart pole problem the number of states is equal to the number of configurations of the 4 discreteized enviromental descriptors, and there are only three actions: move the cart left, right, or keep it still


For a given problem we will denote the state and actions spaces as follows

- Denote the set of all states of the system  $S=\left\{ \sigma_{1},\sigma_{2},...,\sigma_{N}\right\} $ 


- Denote the set of all actions available  $A=\left\{ \alpha_{1},\alpha_{2},...,\alpha_{M}\right\} $


Notice that if we ourselves were to solve any of the examples given so far we would do so sequentially, in discrete steps.  This is precisely how the agent solves these problems as well.  

- In grid world we / the agent move the player one square at a time.  Beginning at a state (location) an action is taken to move the player to a new state.  For taking this action and moving to the new state the agent receives a reward.  This reward is either positive or negative based on whether - in the long run - this helped or hindered it achieivng the goal (reaching the green square).


- In chess we / the agent moves one piece at a time.  Beginning at a state (a specific configuration of the board) an action is taken and a piece on the board is moved, a new state of the system.  For taking this action and moving to the new state the agent receives a reward.  This reward is either positive or negative based on whether - in the long run - this helped or hindered it achieivng the goal (check mating the opponent).


- In cart pole we / the agent keep the cart still or move it with a fixed force left or right.  Beginning at a state (a specific configuration of the 4 system descriptors) an action is taken and the cart is moved, a new state of the system arises.  For taking this action and moving to the new state the agent receives a reward.  This reward is either positive or negative based on whether - in the long run - this helped or hindered it achieivng the goal (keeping the pole in the air as long as possible).


Some notation to describe what occurs at the $k^{th}$ step in solving a problem: at this step an agent begins at a state $s_{k-1} \in S$, takes an action $a_k \in A$ that moves the system to a state $s_k \in S$.

Notice too a key part of each of these descriptions when it comes to an agent solving a given problem: the mechanism by which an agent learns the best action to take in a given state is the reward it recieves.  This reward is positive or negative based on how each action helps or hinders its accomplishment of the goal.

We need notation for the reward an agent recieves at the $k^{th}$ step as well: call this $r_k$.  This is a function of the initial state at this step, the action taken, and the preceeding state - i.e., $r_k = r_k(s_{k-1},a_k,s_k)$

So in solving a problem an agent goes through a sequence of events at each step.  These are: start at a state, take an action, move to a new state, and recieve a corresponding reward.  The first three steps look like

**step 1:** start at state 0 $s_0$ --> take action 1 $a_1$ --> move to state 1 $s_1$ + recieve reward 1 $r_1$

**step 2:** start at state 1 $s_1$ --> take action 2 $a_2$--> move to state 2 $s_2$ + recieve reward 2 $r_2$

**step 3:** start at state 2 $s_2$ --> take action 3 $a_3$ --> move to state 3 $s_3$ + receive reward 3 $r_3$

**step 4:** ...

or in short the first three steps look like

($s_0$, $a_1$, $r_1$), ($s_1$, $a_2$, $r_2$), ($s_2$, $a_3$, $r_3$), ($s_3$,...

What do we want a trained agent to do at each of these steps?  Choose the correct action that maximizes its long term reward - that is the sum of rewards over all the steps. 

# 2  Training the agent by Q learning

So far we have set up the standard RL problems, defined its various components and introduced notation for each, and discussed how these components interact in each step of a problem solving process.  In this Section we will discuss a basic approach to training an agent to take the right actions to maximize its total rewards.

How do we do this - train the agent to take the proper actions at each state to **maximize its total reward**?  

Calling $r_k = r(s_{k-1},a_k,s_k)$ the reward at the $k^{th}$ step by taking action $a_k$ and moving from state $s_{k-1}$ to $s_k$, mathematically we want to maximize the sequence of rewards

$Q(s_0,a_1) = r_1 + r_2 + r_3 + ... $

This is a cost function that we would like to be maximal for every possible initial state $s_0$. 

We cannot expect to directly apply the conventional tools of nonlinear optimization (e.g., stochastic gradient descent) to maximize $Q(s_0,a_1)$, as e.g., both inputs are discrete.  So what can we do?

### A more common Q-learning cost function

A more common cost function used to maximize these rewards dampens later rewards using a controller $\gamma \in [0,1]$ to lessen the effect of future state rewards. 

$Q(s_0,a_1) = r_1 + \gamma^1r_2 + \gamma^2r_3 + ... $

By scaling $\gamma$ up and down we can lessen the contribution of future rewards.  For example

- When we set $\gamma = 0$ then only the first reward remains $r_1$.   Maximizing $Q$ means we maximize the reward given by the first step of the process, for all states.  Our agent learns to take a 'greedy' approach to accomplishing our goal, at each state taking the next step that maximizes the next step reward only.


- When we set $\gamma = 1$ then we all the $\gamma$'s disappear and we have our original cost function. 

##  2.1  Maximizing Q via recursion 

How can we maximize

$Q(s_0,a_1) = r_1 + \gamma^1r_2 + \gamma^2r_3 + ... $

in practice?  

Call this maximum (that is, the largest sum of rewards after recieving $r_1$ from making decision $a_1$ at $s_0$) $Q^*(s_0,a_1)$.  The trick here is that this definition is *recursive*, that is

$Q(s_0,a_1) = r_1 + \gamma Q(s_1,a_2)$

This recursive version is often called **Bellman's equation**. 

While this recursive definition doesn't help us to optimize $Q$ *directly*, we can use it to apply a heuristic in order to *approximately* maximize this quantity.  

In short: we run through the gamat of possible initial states multiple times and update $Q$ by trial-and-error interactions with the enviroment.  

We run a (large number of) simulations - called *episodes* in RL jargon - which en masse help carve out the right set of actions for our agent to take.  

Here's how we do it - we run over a large sequence of episodes where we allow our agent to run the user around towards the goal from various initial states, and update $Q^*$ as we go along.  For each episode we:

------
1.  Initialize $Q$ 
2.  Select a random initial state $s_0$ 
3.  While the goal state has not been reached we 
    -  select a random action at our current state $s_k$
    -  update our approximation 
    
    $Q(s_k,a_{k+1}) = r_k + Q(s_{k+1},a_{k+2})$
    
    -  update state $s_k$ --> $s_{k+1}$ given action $a_k$

-----
 
Eventually, as we cycle through the entire set of initial states through episodes this will converge to the true value of $Q^*$.  Note: importantly, in order for $Q^*$ to be trained properly we **must cycle through every possible state** multiple times.  

We perform Q-learning for our grid world problem in the next Python cell.

In [12]:
# train q learning and lets examine the matrix as it develops
test.qlearn(gamma = 0.5)

q-learning process complete


### 2.2  Representing the Q function as a 2-dimensional matrix

While mathematically the input to $Q(s,a)$ can have many dimensions, being pairs of raw states and actions, the way we compute it via *Bellman's equation* / store it on a computer is via a 2-dimensional matrix representation.  

If we order the set of all possible states, and denote them as $S=\left\{ \sigma_{1},\sigma_{2},...,\sigma_{N}\right\} $, and do the same thing with the set of all possible actions $ A=\left\{ \alpha_{1},\alpha_{2},...,\alpha_{M}\right\} $ (note: this is not an ordering of states / actions as they occur, but of all possible states / actions available to the system), then we represent $Q$ on as computer as the $N \times M$ matrix


$Q=\left[\begin{array}{cccc}
Q\left(\sigma_{0},\alpha_{0}\right) & Q\left(\sigma_{0},\alpha_{1}\right) & \cdots & Q\left(\sigma_{0},\alpha_{M}\right)\\
Q\left(\sigma_{1},\alpha_{0}\right) & \ddots &  & Q\left(\sigma_{1},\alpha_{1}\right)\\
\vdots &  & \ddots & \vdots\\
Q\left(\sigma_{N},\alpha_{0}\right) &  &  & Q\left(\sigma_{N},\alpha_{M}\right)
\end{array}\right]$

In other words, we represent $Q$ as a matrix indexed by action along its columns and state along its rows.

### 2.3  What is the best action per state given a learned Q function?

Given $Q^*(s_k,a)$ learned properly, the agent moves intelligently at a given state $s_k$ by taking the action that maximizes $Q^*$ at this state.  This function - from states to actions - is usually referred to as a **optimal policy function** and denoted as $\pi(s_k)$.  So we then have that

$\pi(s_k) = \underset{a}{\operatorname{argmax}} Q^*(s_k,a_{}) $

Remember, again, that $\pi(s_k)$ is an **action** that takes us from state $s_k$ to some state $s_{k+1}$.  

So in summation - after training $Q^*$ we can start at any initial state $s_0$, and using the optimal policy function $\pi$ we travel from state to state until we reach the goal (or a maximum number of steps are taken).

In the next Python you can initialize the agent at any square on the board, and activating the cell will animate its path (using the optimal policy function) to the goal).

In [9]:
starting_location = [1,1]
test.animate_movement(starting_location)

,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


# 3.  The importance of features

In detailing the pieces of a standard RL problem in the first Section we mentioned 3 components: states, actions, and rewards.  But there is an important $4^{th}$ component to an RL system we did not mention - features.  

In this Section we describe this final RL component and weave features into formal description in Section 2 that led to the Q learning scheme.

## 3.1  What are features?

Features, the $4^{th}$ RL component, play the same role in Reinforcement Learning as they do in machine learning more generally: they characterize input so that a proper relationship between input and output can be learned.

4\.    **Features** are pieces of information that we (humans) design and which characterize each state in the state space.  The agent uses the features from each state to make its decision on how to behave to achieve our goal.  Again, what sort of information to give to our agent **is completely up to us to decide.**  


-  For grid world a fine choice of features are simply the raw state data itself - that is the tuple (the horizontal / vertical coordinates) describing the location of the player.  Another choice: two features 1) the distance from the player to the goal (one feature) and 2) the distance from player to the nearest wall. 


- For the chess problem a natural choice is one set of 64 features, the $i^{th}$ feature indicating which (if any) piece is occupying the $i^{th}$ square of the board (there are 64 squares in total, hence 64 features).  However there are other choices we could make as well.


- For the cart-pole problem a natural set of features are just the four state descriptors (discretized properly).


## 3.2  Formally integrating features into the RL framework

Formally we can denote a set of features extracted from a state $s$ using function notation as $f(s)$.  Note this (potentially vector-valued) feature transformation is often an abstract set of processing steps on the problem enviroment (as in the grid-world or chess examples) and need not solely be a function in the classical mathematical sense (i.e., $f (s) = \text{sin}(s)$ ).  

An AI agent - and the Q learning process - learns based on the features extracted at each state.  This means that in general the input to the $Q$ function are not states and actions as $Q(s,a)$, but features and actions $Q(\text{f}(s),a)$.


$Q$ is still stored and computed as an $N \times M$ matrix, but is the features from each state that are used as input instead of the raw state itself (unless the features are chosen to be the state itself).


$Q=\left[\begin{array}{cccc}
Q\left(f\left(\sigma_{0}\right),\alpha_{0}\right) & Q\left(f\left(\sigma_{0}\right),\alpha_{1}\right) & \cdots & Q\left(f\left(\sigma_{0}\right),\alpha_{M}\right)\\
Q\left(f\left(\sigma_{1}\right),,\alpha_{0}\right) & \ddots &  & Q\left(f\left(\sigma_{1}\right),\alpha_{1}\right)\\
\vdots &  & \ddots & \vdots\\
Q\left(f\left(\sigma_{N}\right),,\alpha_{0}\right) &  &  & Q\left(f\left(\sigma_{N}\right),\alpha_{M}\right)
\end{array}\right]$

## 3.3  What features should you choose 

### Choosing features that provide the best information about your problem states

Many machine learning problems require that you know a substantial amount of information about a given domain in order to make a proper choice of features.  You don't want features that are 'uninformative', that tell you very little about the underlying system you're trying to model.  In classification  - for example -  you want features that sharply distinguish between different classes of data.  Choosing features that fail to do this means that - regardless of your classification algorithm - you won't get good results.  Designing features that accomplish this goal can require serious reflection and work.  

Generally speaking the same holds in the RL setting as well.  If you are working on a complex robotics problem your time spent engineering features could be substantial.  Analog to classification, in the RL setting you want features that sharply distinguish between *states* of your system.  If your features fail to accomplish this you will train a poorly performing agent.  Designing features that accomplish this goal can require serious reflection and work.  

For some RL problems - however - choosing reasonable features is fairly intuitive.  In the toy problems we discussed here - for example - choosing features was straight forward (sometimes we just took the raw state itself as the feature).  In other game related problems the choice can be more intuitive as well - but the more you understand the game the better the features you can devise.

### Choosing features based on a product or academic goal

Another question to consider when choosing features for RL is: what are your intentions?  

- Are they practical?  Do you want to produce a robot with some automatic control feature and sell it to paying customers?  Then you want invest heavily in resources to construct a set of features that will work quickly and effectively with your application.


- Are they academic-focused?  Then your goal may be set by a curiousity of your own or the academic establishment.  One academic question investigated a lot these days is to find a set of features you can effectively apply to solving a large set of visual problems (e.g., convolutional networks and RL Atari AI)


# 4.  Dealing with large state spaces

Notice one major fundamental difference (in terms of the RL components of each) between the chess and cart-pole examples and the grid-world example discussed previously: the size of the state-space.  

- With the chess example: the number of possible configurations of chess pieces on the board is enormous - i.e., the number of states or size of state space - is on the order of $10^{120}$.  That is far larger than the number of atoms in the known universe!


- With the cart-pole example: each of the four descriptors of the enviroment is continuous (or highly discretized) - thus the number of states is infinite.  


In both cases the (very large) size of the state space presents practical problems (while the size of the action space is not a problem).  Why?  Because rememver that in order to learn the $Q^*$ function we have to be able to cycle through *every possible state*.  Practically speaking then even a state space of size $10^{120}$ is far too large for us to be able to learn a representative $Q^*$ effectively.  

Even more practically, remember that so far we have been storing the $Q$ function as a matrix of size $N \times M$, where $N$ and $M$ are the sizes of the state and action spaces respectively.  But the size of $N$ is - for these two problems and for most real world problems (in automatic control and game AI) far too large store in memory!  

In sum the problem here that most real RL problems the **size of the state space** makes computing and storing the $Q$ function/matrix impossible.  Some RL problems can also have issues with large sized action spaces, but this is typically not as problematic (as action spaces can be more easily discretized).

## 3.2  Compactly representing state space of the $Q$ function

We need a way to compactly represent *the vertical or state-space dimension* of $Q$ when we have a very large (and possibly infinite number) of states.  Instead of trying to compute and record every input/output pair - which is impossible once the input space is too large or infinite - we can try the next best thing: represent $Q^*$ as a *parameterized function* and simply fit it to the input/output data.

Does the concept of "fitting a parameterized function to a set of input/output data" sound familiar?  Thats just a fancy way of saying that we'll use **regression** - a fundamental machine learning tool.

## 3.3  Using regression to represent the columns of Q

Instead of storing the entire matrix $Q$, we will substitute a parameterized function for each of its columns - independently.   The $k^{th}$ available action $a_k$ has a corresponding slice of $Q$ (an $N \times 1$ column vector)

$Q_k=\left[\begin{array}{c}
Q\left(\sigma_{0},\alpha_{k}\right)\\
Q\left(\sigma_{1},\alpha_{k}\right)\\
\vdots\\
Q\left(\sigma_{N},\alpha_{k}\right)
\end{array}\right]$

As with the $Q$ matrix itself - the length of this vector is far too long to completely compute (via Belman's equation) or store. 

We can use a parameterized function to approximate the values of this column vector, which we will denote as 

$Q_k(s,\theta) \approx Q_k$

where $\theta$ denotes the parameters of the function.

The simplest - yet still commonly used - is a linear combination of the input features.  



whose parameters we will tune to data.  

### The cart pole example

In [ ]:
import gym
env = gym.make('CartPole-v0')
env.reset()
for _ in range(1000):
    env.render()
    env.step(env.action_space.sample()) # take a random action